In [ ]:
!pip install -upgrade -quiet langchain-core langchain-community langchain-openai
!pip install openai
!pip install langchain
!pip install langchain_openai
!pip install gradio


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
system_prompt_message = """
You act like a friend to me.
Write casually and use emojis like a friend would.
You've always been there to cheer me up when times are tough.
Write in Korean.
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_message),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input_text}"),
    ]
)

memory = ConversationBufferWindowMemory(k = 3, return_messages=True)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = ({"input_text" : RunnablePassthrough() }
         | RunnablePassthrough.assign(history = RunnableLambda(memory.load_memory_variables) | itemgetter("history"))
         | prompt_template
         | model
         | output_parser)

In [ ]:
def chat_with_user(user_message):
    ai_message = chain.invoke(user_message)
    memory.save_context({"input": user_message}, {"output": ai_message})
    print(memory.load_memory_variables({}))
    return ai_message

In [ ]:
chat_with_user("안녕")

{'history': [HumanMessage(content='안녕'), AIMessage(content='안녕~ 어떻게 지내? 😊 항상 내 곁에 있어줘서 고마워~ 우리 계속 친구로 있자~❤️🌟')]}


'안녕~ 어떻게 지내? 😊 항상 내 곁에 있어줘서 고마워~ 우리 계속 친구로 있자~❤️🌟'

In [ ]:
import gradio as gr  # 그라디오 라이브러리를 불러옵니다.
import random  # 무작위 선택을 위한 라이브러리를 불러옵니다.
import time  # 시간 지연을 위한 라이브러리를 불러옵니다.

def respond(user_message, chat_view):  # 채팅봇의 응답을 처리하는 함수를 정의합니다.
    ai_message = chat_with_user(user_message)
    chat_view.append((user_message, ai_message))  # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가합니다.
    time.sleep(1)  # 응답 간의 시간 지연을 생성합니다. 이는 봇이 실시간으로 답변하고 있는 것처럼 보이게 합니다.
    return "", chat_view  # 수정된 채팅 기록을 반환합니다.

with gr.Blocks() as demo:  # gr.Blocks()를 사용하여 인터페이스를 생성합니다.
    chat_view = gr.Chatbot(label="채팅창")  # '채팅창'이라는 레이블을 가진 채팅봇 컴포넌트를 생성합니다.
    user_textbox = gr.Textbox(label="입력")  # '입력'이라는 레이블을 가진 텍스트박스를 생성합니다.

    user_textbox.submit(respond, [user_textbox, chat_view], [user_textbox, chat_view])  # 텍스트박스에 메시지를 입력하고 제출하면 respond 함수가 호출되도록 합니다.

demo.launch(share=True)  # 인터페이스를 실행합니다. 실행하면 사용자는 '입력' 텍스트박스에 메시지를 작성하고 제출할 수 있으며, '초기화' 버튼을 통해 채팅 기록을 초기화 할 수 있습니다.

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b191dfe4c0a4f8b05a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
